In [ ]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt

from factortest.preprocess_data.load_data import load_sp500_tickers, fetch_data_from_yahoo, fetch_fama_french_monthly_factors, generate_monthly_returns_from_daily_price_data
from factortest.estimate.factor_model import estimate_ff3, estimate_capm

import datetime

import warnings

warnings.filterwarnings('ignore')


In [ ]:
companies = load_sp500_tickers()
companies

In [ ]:
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2023, 1, 1)
market_data = fetch_data_from_yahoo(companies, start,end)
market_data

In [ ]:
factors = fetch_fama_french_monthly_factors()
factors

In [ ]:
price_column = 'Adj Close'
ticker_column = 'Symbol'
date_column = 'Date'
monthly_returns = generate_monthly_returns_from_daily_price_data(market_data, price_column, ticker_column, date_column)
monthly_returns

In [ ]:
monthly_returns = monthly_returns.join(factors, on='Date', how='left')
monthly_returns['Excess Return'] = monthly_returns['Monthly Returns'] - monthly_returns['RF']

In [ ]:
monthly_returns.sort_values(by='Date')

In [ ]:
LHS = 'Excess Return'
MARKET = 'Mkt-RF'
FF = ['Mkt-RF', 'SMB', 'HML']

In [ ]:
ff_params_df = estimate_ff3(monthly_returns, left_hand_side_variable=LHS, right_hand_side_variables=FF, ticker_column=ticker_column)
ff_params_df

In [ ]:
capm_params_df = estimate_capm(monthly_returns, left_hand_side_variable=LHS, right_hand_side_variable=MARKET, ticker_column=ticker_column)
capm_params_df

In [ ]:
ticker = 'TSLA'

to_plot = monthly_returns[monthly_returns['Symbol'].isin([ticker])]
to_plot.plot(kind = 'scatter', x = 'Mkt-RF', y = 'Excess Return', figsize=(8,8))
plt.plot(to_plot['Mkt-RF'], capm_params_df['Mkt-RF coef'].filter(like=ticker).values.astype(float) * to_plot['Mkt-RF'] + capm_params_df['intercept coef'].filter(like=ticker).values.astype(float), '-', color='r')
plt.text(-0.15, -1.3, f"P-value of market beta: {capm_params_df['Mkt-RF p-value'].filter(like=ticker).values.item(0):.3}")
plt.show()